# Tensorflow 튜토리얼
## 소개
텐서플로우(TensorFlow)는 기계 학습과 딥러닝을 위해 구글에서 만든 오픈소스 라이브러리입니다. 데이터 플로우 그래프(Data Flow Graph) 방식을 사용하였습니다.

### 데이터 플로우 그래프

데이터 플로우 그래프는 수학 계산과 데이터의 흐름을 노드(Node)와 엣지(Edge)를 사용한 방향 그래프(Directed Graph)로 표현합니다.

![data flow graph](https://www.tensorflow.org/images/tensors_flowing.gif)

노드는 수학적 계산, 데이터 입/출력, 그리고 데이터의 읽기/저장 등의 작업을 수행합니다. 엣지는 노드들 간 데이터의 입출력 관계를 나타냅니다.

엣지는 동적 사이즈의 다차원 데이터 배열(=텐서)을 실어나르는데, 여기에서 텐서플로우라는 이름이 지어졌습니다.

> 텐서(Tensor)는 과학과 공학 등 다양한 분야에서 이전부터 쓰이던 개념입니다. 수학에서는 [임의의 기하 구조를 좌표 독립적으로 표현](http://ghebook.blogspot.kr/2011/06/tensor.html)하기 위한 표기법으로 알려져 있지만, 분야마다 조금씩 다른 의미로 사용됩니다. 여기에서는 *학습 데이터가 저장되는 다차원 배열* 정도로 이해하시면 되겠습니다.

### 특징

텐서플로우는 다음과 같은 특징을 가집니다:

- 데이터 플로우 그래프를 통한 풍부한 표현력
- 코드 수정 없이 CPU/GPU 모드로 동작
- 아이디어 테스트에서 서비스 단계까지 이용 가능
- 계산 구조와 목표 함수만 정의하면 자동으로 미분 계산을 처리 
- Python/C++를 지원하며, [SWIG](http://www.swig.org)를 통해 다양한 언어 지원 가능

*이후의 설명은 [Python](https://www.python.org)을 중심으로 진행하겠습니다.* (pip를 통한 Python3설치는 개발 중으로, Python2 기반으로 하겠습니다.)

> *"구글이 텐서플로우를 오픈소스로 한 것은, 기계 학습이 앞으로 제품과 기술을 혁신하는데 가장 필수적인 요소라고 믿기 때문입니다."      - Google Brain Team*


## 기본 개념 익히기

일단 기본 용어부터 살펴보겠습니다.

### 용어
#### 오퍼레이션(Operation)
그래프 상의 노드는 오퍼레이션(줄임말 *op*)으로 불립니다. 오퍼레이션은 하나 이상의 *텐서*를 받을 수 있습니다. 오퍼레이션은 계산을 수행하고, 결과를 하나 이상의 텐서로 반환할 수 있습니다.

#### 텐서(Tensor)
내부적으로 모든 데이터는 텐서를 통해 표현됩니다. 텐서는 일종의 다차원 배열인데, 그래프 내의 오퍼레이션 간에는 텐서만이 전달됩니다. ([Caffe](http://caffe.berkeleyvision.org)의 [Blob](http://caffe.berkeleyvision.org/tutorial/net_layer_blob.html)과 유사합니다.)

#### 세션(Session)

그래프를 실행하기 위해서는 [세션](https://www.tensorflow.org/versions/master/api_docs/python/client.html#session-management) 객체가 필요합니다. 세션은 오퍼레이션의 실행 환경을 캡슐화한 것입니다.

#### 변수(Variables)
[변수](https://www.tensorflow.org/versions/master/how_tos/variables/index.html)는 그래프의 실행시, 패러미터를 저장하고 갱신하는데 사용됩니다. 메모리 상에서 텐서를 저장하는 버퍼 역할을 합니다.


## Single-layer perceptron으로 XOR구현하기

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
x_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y_data = np.array([[0], [1], [1], [0]])

X = tf.placeholder(tf.float32, shape=[4,2], name="x-input")
Y = tf.placeholder(tf.float32, shape=[4,1], name="y-input")

W = tf.Variable(tf.random_uniform([2, 1], -1.0, 1.0))

# hypothesis
h = tf.matmul(X, W)
hypothesis = tf.div(1., 1.+tf.exp(-h))
#hypothesis = tf.sigmoid(tf.matmul(X, W))

# cost function
cost = -tf.reduce_mean(Y*tf.log(hypothesis) + (1-Y)*tf.log(1-hypothesis))

# Minimize
a = tf.Variable(0.01) # Learning rate, alpha
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)

init = tf.initialize_all_variables()

# Launch the graph.
with tf.Session() as sess:
    sess.run(init)
    
    # Test model
    correct_prediction = tf.equal(tf.floor(hypothesis+0.5), Y)
    
    # Fit the line.
    for step in range(10000):
        sess.run(train, feed_dict={X:x_data, Y:y_data})
        if step % 200 == 0:
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
            accuracy_val = accuracy.eval({X:x_data, Y:y_data})
            print("Accuracy:", accuracy_val)
            print(step, sess.run(cost, feed_dict={X:x_data, Y:y_data}), sess.run(W))
    
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    
    print(sess.run([hypothesis, tf.floor(hypothesis+0.5), correct_prediction, accuracy], feed_dict={X:x_data, Y:y_data}))
    print("Accuracy:", accuracy.eval({X:x_data, Y:y_data}))


Accuracy: 0.5
0 0.694017 [[ 0.11375573]
 [ 0.00809596]]
Accuracy: 0.25
200 0.693612 [[ 0.08849787]
 [-0.00475294]]
Accuracy: 0.25
400 0.693408 [[ 0.06992097]
 [-0.01237414]]
Accuracy: 0.25
600 0.693303 [[ 0.05608314]
 [-0.0165417 ]]
Accuracy: 0.25
800 0.693246 [[ 0.0456293 ]
 [-0.01846074]]
Accuracy: 0.25
1000 0.693213 [[ 0.03761235]
 [-0.0189455 ]]
Accuracy: 0.25
1200 0.693194 [[ 0.03136802]
 [-0.01854263]]
Accuracy: 0.25
1400 0.693181 [[ 0.02642819]
 [-0.01761637]]
Accuracy: 0.25
1600 0.693173 [[ 0.02246104]
 [-0.01640683]]
Accuracy: 0.25
1800 0.693166 [[ 0.01922956]
 [-0.01507   ]]
Accuracy: 0.25
2000 0.693162 [[ 0.016563  ]
 [-0.01370516]]
Accuracy: 0.25
2200 0.693159 [[ 0.01433702]
 [-0.01237355]]
Accuracy: 0.25
2400 0.693156 [[ 0.01246006]
 [-0.01111106]]
Accuracy: 0.25
2600 0.693154 [[ 0.01086374]
 [-0.00993692]]
Accuracy: 0.25
2800 0.693152 [[ 0.00949629]
 [-0.00885951]]
Accuracy: 0.25
3000 0.693151 [[ 0.0083179 ]
 [-0.00788041]]
Accuracy: 0.25
3200 0.69315 [[ 0.00729749]
 [-0.

## Multi-layer perceptron으로 XOR구현하기

In [8]:
x_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y_data = np.array([[0], [1], [1], [0]])

X = tf.placeholder(tf.float32, shape=[4,2], name="x-input")
Y = tf.placeholder(tf.float32, shape=[4,1], name="y-input")

W1 = tf.Variable(tf.random_uniform([2, 2], -1.0, 1.0), name="Weight1")
W2 = tf.Variable(tf.random_uniform([2, 1], -1.0, 1.0), name="Weight2")

b1 = tf.Variable(tf.zeros([2]), name="Bias1")
b2 = tf.Variable(tf.zeros([1]), name="Bias2")

# hypothesis
L2 = tf.sigmoid(tf.matmul(X, W1) + b1)
hypothesis = tf.sigmoid(tf.matmul(L2, W2) + b2)

# cost function
#cost = -tf.reduce_mean(Y*tf.log(hypothesis) + (1-Y)*tf.log(1-hypothesis))
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(hypothesis, Y))

# Minimize
a = tf.Variable(0.5) # Learning rate, alpha
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)

init = tf.initialize_all_variables()

# Launch the graph.
with tf.Session() as sess:
    sess.run(init)
    
    # Test model
    correct_prediction = tf.equal(tf.floor(hypothesis+0.5), Y)
    
    # Fit the line.
    for step in range(20000):
        sess.run(train, feed_dict={X:x_data, Y:y_data})
        if step % 500 == 0:
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
            accuracy_val = accuracy.eval({X:x_data, Y:y_data})
            print("Accuracy:", accuracy_val)
            print(step, sess.run(cost, feed_dict={X:x_data, Y:y_data}),"W1", sess.run(W1),"W2", sess.run(W2))
            
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    
    print(sess.run([hypothesis, tf.floor(hypothesis+0.5), correct_prediction, accuracy], feed_dict={X:x_data, Y:y_data}))
    print("Accuracy:", accuracy.eval({X:x_data, Y:y_data}))



Accuracy: 0.5
0 0.733014 W1 [[ 0.47215295 -0.69309139]
 [ 0.24930049 -0.53286612]] W2 [[ 0.28796157]
 [ 0.3299177 ]]
Accuracy: 0.5
500 0.694023 W1 [[ 0.48714459 -0.71728677]
 [ 0.24278042 -0.56543088]] W2 [[-0.72024435]
 [-0.4035407 ]]
Accuracy: 0.5
1000 0.693487 W1 [[ 0.4895283  -0.75421178]
 [ 0.21315995 -0.61835235]] W2 [[-0.87626064]
 [-0.55592251]]
Accuracy: 0.5
1500 0.693312 W1 [[ 0.49425572 -0.79633206]
 [ 0.1824806  -0.67781311]] W2 [[-0.9608531 ]
 [-0.65095031]]
Accuracy: 0.5
2000 0.693214 W1 [[ 0.50394738 -0.84290397]
 [ 0.15266833 -0.74301791]] W2 [[-1.01804209]
 [-0.72456574]]
Accuracy: 0.5
2500 0.693138 W1 [[ 0.51905942 -0.89447898]
 [ 0.12317244 -0.81463188]] W2 [[-1.06011093]
 [-0.78869474]]
Accuracy: 0.5
3000 0.693067 W1 [[ 0.53987592 -0.9519397 ]
 [ 0.09307891 -0.89366746]] W2 [[-1.09191132]
 [-0.84946865]]
Accuracy: 0.5
3500 0.69299 W1 [[ 0.56687897 -1.01638985]
 [ 0.06138476 -0.9813807 ]] W2 [[-1.11559188]
 [-0.9110899 ]]
Accuracy: 0.5
4000 0.692896 W1 [[ 0.60091496 